In [1]:
from enhanced_hybrid_hhl import RandomQLSP, QPE_preprocessing, QCL_QPE_IBM, QuantumLinearSystemSolver, ideal_preprocessing
from qiskit import transpile, QuantumCircuit
import numpy as np
from qiskit.circuit.library import HamiltonianGate


In [2]:
from qiskit_ibm_provider import IBMProvider
from qiskit.providers.aer import AerSimulator

provider = IBMProvider(instance='ibm-q-ncsu/nc-state/amplitude-estima')
torino = provider.get_backend('ibm_torino')
simulator = AerSimulator()
emulator = AerSimulator().from_backend(torino)

In [3]:
problem = RandomQLSP(2, 5)
clock = 3

ideal_lists = ideal_preprocessing(problem)

max_eigenvalue = max(ideal_lists[0], key=abs)

scale = abs((0.5-2**-clock)/max_eigenvalue)

ideal_lists

([-2.4804041152739553,
  -1.6624312014267035,
  -0.654341361893473,
  1.3677182966260804],
 [0.24540129943736844,
  0.14770664141222953,
  0.8449938816744625,
  0.45160413001864236])

In [4]:
def get_results_preprocessing(circ):
    transp = transpile(circ, simulator)
    result = simulator.run(transp, shots=4000).result()
    return result

def fromat_results_preprocessing(result):
    
    min_prob = 2**(-clock)

    counts=result.get_counts()

    tot = sum(counts.values())
    result_dict = {(int(key,2) if key[0]=='0' else (int(key,2) - (2**(len(key))))) : value / tot for key, value in counts.items()}


    eigenvalue_list = [eig/(scale*2**(clock)) for eig in result_dict.keys() if result_dict[eig] > min_prob]
    eigenbasis_projection_list = [result_dict[eig] for eig in result_dict.keys() if result_dict[eig] > min_prob]
    return eigenvalue_list, eigenbasis_projection_list

In [5]:
def construct_QCL_QPE_circuit(problem, scale, clock):
    '''Constructs QPE_QCL circuit for a given time parameter'''
    qubits = int(np.log2(problem.A_matrix.shape[0]))
    circ = QuantumCircuit(qubits+1,clock)
    circ.prepare_state(problem.b_vector.T.tolist()[0], list(range(1, circ.num_qubits)))
    for clbit in range(clock):
        if clbit!=0:
            circ.initialize([1,0],[0])
        circ.h(0)
        power=2**(clock-clbit-1)
        ham = HamiltonianGate(problem.A_matrix,-2*np.pi*scale).power(power).control()
        circ.append(ham, circ.qubits)
        
        for i in reversed(range(clbit)):
            #tb = QuantumCircuit(1,1)
            if i < clock:   
                N = (2**(i+2))
                #tb.p(np.pi*2/N,0)
                control = clbit-i-1
                with circ.if_test((control,1)) as passed:
                    circ.p(-np.pi*2/N,0)
            #circ.if_else((control,1), true_body=tb, false_body=None, qubits=[0], clbits=[control])
        circ.h(0)
        circ.measure(0,[clbit])
    return circ

In [6]:

circ = construct_QCL_QPE_circuit(problem, scale, clock)

baseline_result = get_results_preprocessing(circ=circ)
baseline_lists = fromat_results_preprocessing(baseline_result)

baseline_lists

([2.8174897026274968, -1.8783264684183314, -0.9391632342091657],
 [0.32925, 0.24175, 0.17075])

In [5]:
qcl_qpe = QCL_QPE_IBM(clock, simulator, max_eigenvalue=max_eigenvalue)
qcl_qpe_lists = qcl_qpe.estimate(problem)
qcl_qpe_lists

([-0.8268013717579851, 1.6536027435159701], [0.62425, 0.14375])

In [6]:
qpe = QPE_preprocessing(num_eval_qubits=clock, max_eigenvalue=max_eigenvalue, get_result=get_results_preprocessing)
qpe_lists = qpe.estimate(problem)
qpe_lists

([-0.8268013717579851, 1.6536027435159701], [0.62575, 0.1435])

In [17]:
from qiskit_ionq.ionq_provider import IonQProvider
ionq_provider = IonQProvider(token='W8fkNnlIqaWDP83QCPTnP5HjoELVXMbP')
ionq_simulator = ionq_provider.get_backend('qpu.harmony')

def get_ionq_results_preprocessing(circ):
    transp = transpile(circ, ionq_simulator)
    print(transp.depth())
    result = ionq_simulator.run(transp, shots=4000, noise='qpu.harmony').result()
    return result


In [18]:
test_ionq = QuantumCircuit(1,1)
test_ionq.x(0)
test_ionq.measure(0,0)

ionq_simulator # .run(test_ionq, noise='qpu.harmony')

<IonQQPUBackend('ionq_qpu.harmony')>

In [ ]:
circ = construct_QCL_QPE_circuit(problem, scale, clock)

ionq_qcl_result = get_results_preprocessing(circ=circ)
ionq_qcl_lists = fromat_results_preprocessing(ionq_qcl_result)
ionq_qcl_lists

TranspilerError: "The control-flow construct 'if_else' is not supported by the backend."

In [15]:
ionq_qpe = QPE_preprocessing(clock, max_eigenvalue, get_ionq_results_preprocessing)
ionq_qpe_lists = ionq_qpe.estimate(problem)
ionq_qpe_lists

302


([2.8174897026274968, -1.8783264684183314, -0.9391632342091657],
 [0.2455, 0.32425, 0.16475])